In [1]:
lines = []
with open('./data/train.csv', 'r') as file:
    for i,line in enumerate(file):
        if i < 350000:
            lines.append(line)
        else:
            break
with open('./data/short_train.csv', 'w') as file:
    for line in lines:
        file.write(line)



In [2]:
import pandas as pd
file_name = "data/short_train.csv"
file_name_output = "short_train_no_duplicates.csv"

df = pd.read_csv(file_name)

# Notes:
# - the `subset=None` means that every column is used 
#    to determine if two rows are different; to change that specify
#    the columns as an array
# - the `inplace=True` means that the data structure is changed and
#   the duplicate rows are gone  
df.drop_duplicates(subset=None, inplace=True)

# Write the results to a different file
df.to_csv(file_name_output, index=False)


In [5]:
import pandas as pd
data = pd.read_csv('data/short_train.csv')
data.drop_duplicates(subset=None, inplace = True) 
data.to_csv('jiaiuoehuiog94.csv')


In [1]:
class User:
    def __init__(self, id, bundles):
        self.id = id
        self.bundles = bundles

class QuestionBundle:
    def __init__(self, id, questions):
        self.id = id
        self.questions = questions

class Question:
    def __init__(self, id, answered_correctly, elapsed_time, had_explanation):
        self.id = id
        self.answered_correctly = answered_correctly
        self.elapsed_time = elapsed_time
        self.had_explanation = had_explanation


Map JSON to python object

In [2]:
import json

#path = 'data/slim_train.json'
path = 'data/train.json'
users = []
with open(path, 'r') as file:
    for i, line in enumerate(file):
        j_object = json.loads(line)
        
        users.append(User(
            id=j_object['Id'],
            bundles=[QuestionBundle(
                id=bundle['Id'],
                questions=[Question(
                    id=question['Id'],
                    answered_correctly=question['AnsweredCorrectly'],
                    elapsed_time=question['ElapsedTime'],
                    had_explanation=question['HadExplanation']
                ) for question in bundle['Questions']]
            ) for bundle in j_object['Bundles']]
        ))

Shift `prior_question` columns upwards for each user

In [3]:
for user in users:
    bundles = user.bundles

    for i in range(len(bundles) - 1):
        current_bundle = bundles[i]
        current_questions = current_bundle.questions

        next_bundle = bundles[i + 1]
        next_questions = next_bundle.questions

        for k in range(len(current_questions)):
            current_questions[k].elapsed_time = next_questions[0].elapsed_time
            current_questions[k].had_explanation = next_questions[0].had_explanation

    last_index = len(bundles) - 1
    last_questions = bundles[last_index].questions
    for i in range(len(last_questions)):
        last_questions[i].elapsed_time = 0.0
        last_questions[i].had_explanation = None



In [27]:
# Test - print the two first users as json
users = users[0:2]
print(json.dumps(users, default=lambda x: x.__dict__, indent=True))

[
 {
  "id": 115,
  "bundles": [
   {
    "id": 1,
    "questions": [
     {
      "id": 5692,
      "answered_correctly": true,
      "elapsed_time": 37,
      "had_explanation": false
     }
    ]
   },
   {
    "id": 2,
    "questions": [
     {
      "id": 5716,
      "answered_correctly": true,
      "elapsed_time": 55,
      "had_explanation": false
     }
    ]
   },
   {
    "id": 0,
    "questions": [
     {
      "id": 128,
      "answered_correctly": true,
      "elapsed_time": 19,
      "had_explanation": false
     }
    ]
   },
   {
    "id": 3,
    "questions": [
     {
      "id": 7860,
      "answered_correctly": true,
      "elapsed_time": 11,
      "had_explanation": false
     }
    ]
   },
   {
    "id": 4,
    "questions": [
     {
      "id": 7922,
      "answered_correctly": true,
      "elapsed_time": 5,
      "had_explanation": false
     }
    ]
   },
   {
    "id": 5,
    "questions": [
     {
      "id": 156,
      "answered_correctly": true,
      "elapsed

In [6]:
with open('data/slim_train_shifted.csv', 'w') as file:
    file.write('user_id,question_id,bundle_id,answered_correctly,elapsed_time,had_explanation\n')
    for user in users:
        for bundle in user.bundles:
            for question in bundle.questions:
                file.write(f'{user.id},{question.id},{bundle.id},{1 if question.answered_correctly else 0},{question.elapsed_time},{-1 if question.had_explanation is None else 1 if question.had_explanation else 0}\n')